In [2]:
import json
from prettytable import PrettyTable

In [1]:
# Read the jsonl file and convert it to a JSON list
def jsonl_to_json_list(jsonl_file_path):
    json_list = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line.strip())  # Parse each line as JSON
            json_list.append(json_obj)
    
    return json_list

# Save the JSON list to a file
def save_as_json(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(json_list, outfile, indent=4)

# Example usage:
json_list = jsonl_to_json_list('./data/medqa.jsonl')
save_as_json(json_list, './data/medqa.json')


In [7]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def deduplicate_data(data):
    seen = set()
    deduplicated_data = []
    for item in data:
        idx = item['idx']
        if idx not in seen:
            deduplicated_data.append(item)
            seen.add(idx)
    return deduplicated_data

def calculate_accuracy(data):
    correct_predictions = 0
    total_predictions = len(data)
    for item in data:
        if item['label'] == item['prediction']:
            correct_predictions += 1
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

models = ['gpt-4o-mini', 'gpt-4o']
tasks = ['sampled_50', 'sampled_50_hard', 'sampled_50_5options']
difficulties = ['adaptive']

table = PrettyTable()
table.field_names = ["Model", "Task", "Difficulty", "Accuracy", "Total Number"]

for model in models:
    for task in tasks:
        for difficulty in difficulties:
            file_path = f'./output/{model}_medqa_{task}_{difficulty}.json'
            data = load_json(file_path)
            deduplicated_data = deduplicate_data(data)
            accuracy = calculate_accuracy(deduplicated_data)
            table.add_row([model, task, difficulty, accuracy * 100, len(deduplicated_data)])

print(table)

+-------------+-----------------+------------+----------+--------------+
|    Model    |       Task      | Difficulty | Accuracy | Total Number |
+-------------+-----------------+------------+----------+--------------+
| gpt-4o-mini |    sampled_50   |  adaptive  |   80.0   |      50      |
| gpt-4o-mini | sampled_50_hard |  adaptive  |   36.0   |      50      |
|    gpt-4o   |    sampled_50   |  adaptive  |   84.0   |      50      |
|    gpt-4o   | sampled_50_hard |  adaptive  |   68.0   |      50      |
+-------------+-----------------+------------+----------+--------------+
